In [4]:
#Without UI
import pandas as pd
import joblib
import datetime

# 1. Load Preprocessed Data, Features, and Models
try:
    df_imputed = joblib.load("PB003_preprocessed_df.joblib")
    features_pm25 = joblib.load("PB003_features_pm25.joblib")
    features_pm10 = joblib.load("PB003_features_pm10.joblib")
    model_pm25 = joblib.load("trained_model_pm25.joblib")
    model_pm10 = joblib.load("trained_model_pm10.joblib")
except FileNotFoundError:
    print("Error: One or more required files not found. Make sure you have run the training script first.")
    exit()

# 2. Get User Input for Date and Hour
while True: # Keep asking until valid input is provided
    try:
        date_str = input("Enter date (YYYY-MM-DD): ")
        selected_date = pd.to_datetime(date_str).date()
        hour = int(input("Enter hour (0-23): "))
        if 0 <= hour <= 23:
            break  # Valid input, exit loop
        else:
            print("Invalid hour. Please enter a value between 0 and 23.")

    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")




# 3. Create Input Data for Prediction
selected_datetime = datetime.datetime.combine(selected_date, datetime.time(hour, 0)) #Create datetime object

input_data = pd.DataFrame({'From Date': [selected_datetime]})
input_data['Hour'] = hour
input_data['DayOfWeek'] = selected_date.weekday()
input_data['Month'] = selected_date.month

# Populate other features (using overall averages)
for feature in features_pm25:
    if feature not in ['Hour', 'DayOfWeek', 'Month']:
        input_data[feature] = df_imputed[feature].mean()

for feature in features_pm10:
    if feature not in ['Hour', 'DayOfWeek', 'Month', *features_pm25]:
        input_data[feature] = df_imputed[feature].mean()

# 4. Make Predictions
X_input_pm25 = input_data[features_pm25]
pm25_prediction = model_pm25.predict(X_input_pm25)[0]

X_input_pm10 = input_data[features_pm10]
pm10_prediction = model_pm10.predict(X_input_pm10)[0]

# 5. Classify AQI
def classify_aqi(pm_value): # Classify based on pm2.5 AQI breakpoints
    if pm_value <= 30:
        return "Good"
    elif pm_value <= 60:
        return "Moderate"
    elif pm_value <= 90:
        return "Poor"
    elif pm_value <= 120:
        return "Very Poor"
    else:
        return "Severe"



pm25_aqi_category = classify_aqi(pm25_prediction)
pm10_aqi_category = classify_aqi(pm10_prediction)

# 6. Print Results
print("\n--- Predictions for", selected_date, "at", hour, ":00 ---")  # Cleaner output
print(f"PM2.5 Prediction: {pm25_prediction:.2f} µg/m³ (AQI: {pm25_aqi_category})")
print(f"PM10 Prediction: {pm10_prediction:.2f} µg/m³ (AQI: {pm10_aqi_category})")

Enter date (YYYY-MM-DD):  2001-01-11
Enter hour (0-23):  1



--- Predictions for 2001-01-11 at 1 :00 ---
PM2.5 Prediction: 52.84 µg/m³ (AQI: Moderate)
PM10 Prediction: 136.42 µg/m³ (AQI: Severe)


In [2]:
#With UI
import tkinter as tk
from tkinter import ttk
import pandas as pd
import joblib
import datetime

# 1. Load Preprocessed Data, Features, and Models
try:
    df_imputed = joblib.load("PB003_preprocessed_df.joblib")
    features_pm25 = joblib.load("PB003_features_pm25.joblib")
    features_pm10 = joblib.load("PB003_features_pm10.joblib")
    model_pm25 = joblib.load("trained_model_pm25.joblib")
    model_pm10 = joblib.load("trained_model_pm10.joblib")
except FileNotFoundError:
    print("Error: Required files not found. Make sure 'training.py' has been run.")
    exit()

# 2. Prediction Function
def predict_aqi():
    try:
        selected_date = datetime.datetime.strptime(date_entry.get(), "%Y-%m-%d").date()
        hour = int(hour_entry.get())

        selected_datetime = datetime.datetime.combine(selected_date, datetime.time(hour, 0))
        input_data = pd.DataFrame({'From Date': [selected_datetime]})
        input_data['Hour'] = hour
        input_data['DayOfWeek'] = selected_date.weekday()
        input_data['Month'] = selected_date.month

        for feature in features_pm25:
            if feature not in ['Hour', 'DayOfWeek', 'Month']:
                input_data[feature] = df_imputed[feature].mean()
        for feature in features_pm10:
            if feature not in ['Hour', 'DayOfWeek', 'Month', *features_pm25]:
                input_data[feature] = df_imputed[feature].mean()

        X_input_pm25 = input_data[features_pm25]
        pm25_prediction = model_pm25.predict(X_input_pm25)[0]

        X_input_pm10 = input_data[features_pm10]
        pm10_prediction = model_pm10.predict(X_input_pm10)[0]

        def classify_aqi(pm_value):
            if pm_value <= 30:
                return "Good"
            elif pm_value <= 60:
                return "Moderate"
            # ... (add other AQI categories as needed)
            else:
                return "Severe"

        pm25_aqi_category = classify_aqi(pm25_prediction)
        pm10_aqi_category = classify_aqi(pm10_prediction)

        pm25_label.config(text=f"PM2.5: {pm25_prediction:.2f} µg/m³ (AQI: {pm25_aqi_category})")
        pm10_label.config(text=f"PM10: {pm10_prediction:.2f} µg/m³ (AQI: {pm10_aqi_category})")

    except ValueError:
        pm25_label.config(text="Invalid input. Please enter date as YYYY-MM-DD and hour as a number.")
        pm10_label.config(text="Invalid input. Please enter date as YYYY-MM-DD and hour as a number.")
    except Exception as e:
        pm25_label.config(text=f"An error occurred: {e}")
        pm10_label.config(text=f"An error occurred: {e}")


# --- Tkinter UI ---
root = tk.Tk()
root.title("Ludhiana Air Quality Prediction")


# Date Input
date_label = ttk.Label(root, text="Enter Date (YYYY-MM-DD):")
date_label.grid(row=0, column=0, padx=5, pady=5)
date_entry = ttk.Entry(root)
date_entry.grid(row=0, column=1, padx=5, pady=5)


# Hour Input
hour_label = ttk.Label(root, text="Enter Hour (0-23):")
hour_label.grid(row=1, column=0, padx=5, pady=5)
hour_entry = ttk.Entry(root)
hour_entry.grid(row=1, column=1, padx=5, pady=5)



# Predict Button
predict_button = ttk.Button(root, text="Predict", command=predict_aqi)
predict_button.grid(row=2, column=0, columnspan=2, pady=10)

# Output Labels
pm25_label = ttk.Label(root, text="")
pm25_label.grid(row=3, column=0, columnspan=2, pady=5)
pm10_label = ttk.Label(root, text="")
pm10_label.grid(row=4, column=0, columnspan=2, pady=5)



# Author Information
author_label = ttk.Label(root, text="Built by: Aryamann Sharma, Madhav Rana, Niyamat")
author_label.grid(row=5, column=0, columnspan=2, pady=10)



root.mainloop()